In [1]:
from array import array
import math
import numpy as np # numpy is only used to check implementation
import pickle

In [2]:
def dot(left, right):
    return math.fsum(l * r for l, r in zip(left, right))
def sigmoid(X):
    return 1 / (1 + math.exp(-X))
def label_array(iterable):
    return array('b', list(iterable))
def double_array(iterable):
    return array('d', list(iterable))
def index_column(matrix, index):
    return array(matrix[0].typecode, [row[index] for row in matrix])
def index_columns(matrix, lower, upper):
    return tuple(row[lower : upper] for row in matrix)
def multiply(left, right):
    # element-wise
    return tuple(double_array(l * r for l, r in zip(left_row, right_row)) for left_row, right_row in zip(left, right))
def matrix_dot(left, right):
    return \
        tuple(double_array(dot(row, index_column(right, index)) for index in range(len(right[0]))) for row in left)
def transpose(matrix):
    return tuple(index_column(matrix, i) for i in range(len(matrix[0])))

In [3]:
def load_data(file):
    return tuple(double_array(map(float, line.split(','))) for line in open(file, 'r'))
def load_labels(file):
    return label_array(map(lambda i : int(i) - 1, open(file, 'r')))
def load_dataset():
    return load_data('data.csv'), load_labels('data-labels.csv')
def load_weights():
    return transpose(load_data('theta1.csv')), transpose(load_data('theta2.csv'))

In [4]:
data, labels = load_dataset()
theta1, theta2 = load_weights()

In [5]:
def unpack(theta):
#     return index_columns(theta, 1, len(theta[0])), index_column(theta, 0)
    return theta[1:], theta[0]
def neuron_forward(data, weight, bias, activate=sigmoid):
    '''
    Only to fulfill the requirement of question 1.
    The function is not going to be called because computation of neural networks can be further verctorized.
    '''
    return activate(dot(data, weight) + bias)
def layer_forward(data, weight, bias, activate=sigmoid):
    product = matrix_dot(data, weight)
    linear = tuple(double_array(map(lambda X : X + bias[i], index_column(product, i))) for i in range(len(product[0])))
    nonlinear = tuple(double_array(map(activate, row)) for row in transpose(linear))
    return nonlinear
def network_forward(data, theta1, theta2, activate=sigmoid):
    weight1, bias1 = unpack(theta1)
    activations = layer_forward(data, weight1, bias1, activate)
    weight2, bias2 = unpack(theta2)
    scores = layer_forward(activations, weight2, bias2, activate=lambda X : X)
    cache = (data, activations, weight2)
    return scores, cache
def predict(scores):
    return double_array(row.index(max(row)) for row in scores)
def n_errors(prediction, labels):
    return sum(prediction != label for prediction, label in zip(predictions, labels))
def error_rate(prediction, labels):
    return n_errors(prediction, labels) / len(labels)

In [6]:
# classification error
scores, network_cache = network_forward(data, theta1, theta2)
predictions = predict(scores)
error_rate(predictions, labels)

0.0248

In [7]:
# negative log likelihood loss (convert class scores to probability via softmax function)
def nll_loss(scores, labels):
    N = len(scores)
    maximums = double_array(map(max, scores))
    normalized_scores = \
        tuple(double_array(map(lambda score : score - maximum, row)) for row, maximum in zip(scores, maximums))
    exponents = tuple(double_array(map(math.exp, row)) for row in normalized_scores)
    totals = double_array(math.fsum(row) for row in exponents)
    probabilities = double_array(row[label] / total for row, label, total in zip(exponents, labels, totals))
    negative_log = double_array(map(lambda X : 0 - math.log(X), probabilities))
    loss = math.fsum(negative_log) / N
    cache = (scores, labels, exponents, totals)
    return loss, cache

In [8]:
# compute loss
loss, loss_cache = nll_loss(scores, labels)
loss

0.08688856037475026

In [9]:
def d_nll_loss(cache):
    # gradient w.r.t. class scores
    scores, labels, exponents, totals = cache
    N = len(scores)
    d_scores = tuple(double_array(map(lambda X : X / (N * total), row)) for row, total in zip(exponents, totals))
    for row, label in zip(d_scores, labels): row[label] -= 1 / N
    maximum_indices = map(lambda vector : vector.index(max(vector)), scores)
    for row, index in zip(d_scores, maximum_indices): row[index] -= math.fsum(row)
    return d_scores

In [10]:
# load internal results of vecterized implementation and use those results to check non-vectorized implementation
cache = pickle.load(open('internal', 'rb'))
_scores, _predictions, _d_scores, _d_theta1, _d_theta2 = cache

In [11]:
def relative_error(left, right):
    return np.max(np.abs(left - right) / (np.abs(left) + np.abs(right)))

In [12]:
# compute gradient w.r.t. class scores
d_scores = d_nll_loss(loss_cache)

In [13]:
# compare results yielded by vecterized and non-vecterized implementations
relative_error(np.array(d_scores), _d_scores)

0.62125377330312859

In [14]:
def network_backward(d_scores, cache):
    # gradient w.r.t. parameters
    data, activations, weight2 = cache
    d_bias2 = double_array(sum(index_column(d_scores, i)) for i in range(len(d_scores[0])))
    d_weight2 = matrix_dot(transpose(activations), d_scores)
    d_activations = matrix_dot(d_scores, transpose(weight2))
    d_pre_activations = multiply(
        d_activations,
        multiply(activations, tuple(double_array(map(lambda X : 1 - X, row)) for row in activations)),
    )
    d_bias1 = double_array(sum(index_column(d_pre_activations, i)) for i in range(len(d_pre_activations[0])))
    d_weight1 = matrix_dot(transpose(data), d_pre_activations)
    return d_weight1, d_bias1, d_weight2, d_bias2

In [15]:
# compute gradient w.r.t. parameters
d_weight1, d_bias1, d_weight2, d_bias2 = network_backward(d_scores, network_cache)

In [16]:
def pack(weights, biases):
    # assemble weight and bias into theta
    return transpose(tuple(double_array((bias,)) + row for bias, row in zip(biases, transpose(weights))))

In [17]:
# check gradient w.r.t. theta 1
d_theta1 = pack(d_weight1, d_bias1)
relative_error(np.array(d_theta1), _d_theta1)

/home/alex/python_official/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


nan

In [18]:
# check gradient w.r.t. theta 2
d_theta2 = pack(d_weight2, d_bias2)
relative_error(np.array(d_theta2), _d_theta2)

1.0

It appears that gradient computed via Python arithmetic does not match gradient computed via numpy precisely. However, the statistics of Python gradients matches those of numpy gradients. (Please refer to the other notebook for statistics of numpy gradient)

In [19]:
# statistics w.r.t. theta 1
abs_d_theta1 = np.abs(d_theta1)
mean = np.mean(abs_d_theta1)
median = np.median(abs_d_theta1)
std = np.std(abs_d_theta1)
print('mean = %f' % mean)
print('median = %f' % median)
print('standard deviation = %f' % std)

mean = 0.000115
median = 0.000033
standard deviation = 0.000188


In [20]:
# statistics w.r.t. theta 2
abs_d_theta2 = np.abs(d_theta2)
mean = np.mean(abs_d_theta2)
median = np.median(abs_d_theta2)
std = np.std(abs_d_theta2)
print('mean = %f' % mean)
print('median = %f' % median)
print('standard deviation = %f' % std)

mean = 0.000514
median = 0.000379
standard deviation = 0.000443
